In [1]:
import cv2 
import numpy as np

from sklearn.metrics import pairwise


In [2]:
background = None

accumulated_weight = 0.5  #acculmulated ağılık

#roi reigion of interest


roi_top = 20
roi_bottom = 300
roi_right = 300
roi_left = 600 #kare köşeleri roi



In [3]:
#average background valueyu bulan function
def calc_accum_avg(frame,accumulated_weight):
    
    global background #None olarak başlıyor
    
    if background is None: #gelen frame buraya passlıyacak
        background = frame.copy().astype('float')
        return None
    
    #None değilse değeri varsa yapılacak else
    cv2.accumulateWeighted(frame,background,accumulated_weight) #çalışan ortalama frame değerine biriktiriyor
    
    #bu ağırlığı returnlemiyoruz global arka plana etki ediyor

In [4]:
#Eli segmentleme 

#elin köşelerini bulup segmentliyeceğiz 

def segment(frame,threshold=25):
    
    diff = cv2.absdiff(background.astype('uint8'),frame) #frame ile arkaplanın farkını hesaplayan
                       
    ret,thresholded = cv2.threshold(diff,threshold,255,cv2.THRESH_BINARY) #THRESHOLD = MİN 
                       
    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)#konturları bulma
     
    if len(contours) == 0:
            return None     #kontor uzunluğu kontrolü 
                       
    else:  
                       #en büyük kontor el olması gerektiği için öyle varsayarak else
     hand_segment = max(contours,key=cv2.contourArea)              
     return (thresholded,hand_segment) 

In [5]:
#Convex Hull kullanarak ellerin parmaklarını sayma

In [6]:
#VER 4 ÜSTÜ FİX CHAT GPT  VERSİYON 3 ÜSTÜ İÇİN ENTEGRE KOD 

def count_fingers(thresholded, hand_segment):
    # hand segment numpy array
    hand_segment = np.array(hand_segment)
    if len(hand_segment) == 0:
        return 0
    
    # Compute the convex hull of the hand segment
    try:
        hull = cv2.convexHull(hand_segment, returnPoints=False)
        if len(hull) == 0:
            return 0
        
        # Compute convexity defects
        defects = cv2.convexityDefects(hand_segment, hull)
        if defects is None:
            return 0
    except cv2.error as e:
        print(f"OpenCV error: {e}")
        return 0

    # Initialize the number of fingers
    num_fingers = 0

    # Process convexity defects
    for i in range(defects.shape[0]):
        s, e, f, d = defects[i, 0]
        if d > 10000:  
            num_fingers += 1
    
    return num_fingers


In [7]:
#Kamerayı açma

cam = cv2.VideoCapture(0)

num_frames = 0

while True:
    
    
    ret,frame = cam.read()
    
    frame_copy = frame.copy()
    
    roi = frame[roi_top:roi_bottom,roi_right:roi_left]
    
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    
    gray = cv2.GaussianBlur(gray,(7,7),0)
    
    if num_frames < 60:
        calc_accum_avg(gray,accumulated_weight)
        
        if num_frames <=59:
            cv2.putText(frame_copy,'WAIT GETTING BACKGROUND',(200,300),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
            cv2.imshow('Finger Count',frame_copy)
            
    else:
        
        hand = segment(gray)
        
        if hand is not None:
            
            thresholded, hand_segment = hand
            #gerçek elin etrafına konturlar çiziyor canlı video görüntüsünde
            cv2.drawContours(frame_copy,[hand_segment+(roi_right,roi_top)],-1,(255,0,0),5)
            #parmakları sayma
            fingers = count_fingers(thresholded, hand_segment)
            #sayım sonucunu displayleme
            cv2.putText(frame_copy,str(fingers),(70,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            
            cv2.imshow('Thresholded',thresholded)
    cv2.rectangle(frame_copy,(roi_left,roi_top),(roi_right,roi_bottom),(0,0,255),5)#dışarıda kare çizicez roi ye  
    num_frames += 1
    
    cv2.imshow('Finger Count',frame_copy)
    
    k = cv2.waitKey(1) & 0xFF
    
    if k == 27:
        break
cam.release()
cv2.destroyAllWindows()


OpenCV error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\convhull.cpp:360: error: (-5:Bad argument) The convex hull indices are not monotonous, which can be in the case when the input contour contains self-intersections in function 'cv::convexityDefects'

OpenCV error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\convhull.cpp:360: error: (-5:Bad argument) The convex hull indices are not monotonous, which can be in the case when the input contour contains self-intersections in function 'cv::convexityDefects'

OpenCV error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\convhull.cpp:360: error: (-5:Bad argument) The convex hull indices are not monotonous, which can be in the case when the input contour contains self-intersections in function 'cv::convexityDefects'

OpenCV error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\convhull.cpp:360: error: (-5:Bad argument) The 

In [8]:
import cv2
print(cv2.__version__)


4.10.0
